In [1]:
from dataset.dataset import create_img_from_dir

In [3]:
data_dir = "./data/mit_bih/"
save_dir = "./data/beats_img/"

In [ ]:
create_img_from_dir(data_dir, save_dir)

  0%|          | 0/1888 [00:00<?, ?it/s]

1/48


  0%|          | 0/2295 [00:00<?, ?it/s]

2/48


  0%|          | 0/2092 [00:00<?, ?it/s]

...

100%|██████████| 2272/2272 [03:33<00:00, 10.65it/s] 


13/48


100%|██████████| 2309/2309 [01:47<00:00, 21.44it/s]


14/48


 63%|██████▎   | 1771/2790 [03:45<01:34, 10.76it/s] 

In [1]:
from dataset.dataset import load_files

In [4]:
train, valid, test = load_files(save_dir)

In [5]:
print(len(train))
print(len(valid))
print(len(test))

22448
4810
4815


In [6]:
from pprint import pprint

# pprint(test)

In [7]:
from dataset.data_augmentation import augmentated_filenames2

In [8]:
train = augmentated_filenames2(train)

In [9]:
# pprint(train[:20])

In [10]:
from cnn.cnn import load_dataset, image_to_array

Using TensorFlow backend.
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/congyu/.virtua

In [11]:
import cv2

In [12]:
import os

In [13]:
example_img = os.path.join(save_dir, "NOR", train[0])

In [14]:
img = cv2.imread(example_img, cv2.IMREAD_GRAYSCALE)

In [15]:
img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_LANCZOS4)

In [16]:
img = image_to_array(example_img, (64, 64), True, True)

In [21]:
data_gen_train = load_dataset(train, save_dir, 
32, 
(64, 64),
random_crop=True,
random_rotate=True,
flip=True)

In [24]:
for data in data_gen_train:
    print(data[0].shape)
    cv2.imshow("h", data[0][0])
    break

(32, 64, 64, 1)


In [1]:
from cnn.cnn import training

Using TensorFlow backend.
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/congyu/.virtua

In [2]:
model = training()





Epoch 1/3
 474/7015 [=>............................] - ETA: 12:00 - loss: 1.4233 - accuracy: 0.6396

KeyboardInterrupt: 